In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

In [2]:
url = 'https://www.thegioididong.com/the-nho-dien-thoai'

driver = webdriver.Chrome()
driver.get(url)

### Hiển thị tất cả sản phẩm

In [3]:
show_more_elm = driver.find_element(By.XPATH, '//*[@id="categoryPage"]/div[1]/div[2]/a')
show_more_elm

<selenium.webdriver.remote.webelement.WebElement (session="6925485afae1b5b1a944a0cd1ebb9c79", element="f.67E07BF121A24EFA1A412CE255BCB652.d.EDCCA5255A58F7748F54384D04E48860.e.156")>

In [4]:
while True:
    try:
        show_more_elm.click()
        sleep(2)
    except:
        print('We are at the end of the page!')
        break

We are at the end of the page!


### Lấy danh sách sản phẩm

In [5]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

16

#### Tạo Func lấy thông 1 tin sản phẩm

In [6]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [7]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAllTheNho(product_list):
    for i in range(0, len(product_list)):
        product_info = get_product_info(product_list[i])
        product_list[i].click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            try:
                read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[9]/div[2]/div/div/div[6]/div/a')
            except:
                read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[11]/div[2]/div/div/div[6]/div/a')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        all_reviews.append({
                            'product_name': product_info['name'],
                            'product_price': product_info['price'],
                            'review': review.text
                        })

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                all_reviews.append({
                                    'product_name': product_info['name'],
                                    'product_price': product_info['price'],
                                    'review': review.text
                                })
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')

In [8]:
getReviewAllTheNho(product_list)

Không còn trang tiếp theo!
Không còn trang tiếp theo!
Không còn trang tiếp theo!
Không còn trang tiếp theo!
Không còn trang tiếp theo!
Không còn trang tiếp theo!


In [9]:
df = pd.DataFrame(all_reviews)
df.to_csv('D:\CrawlData\Sentiment_Review\data\cmt_the_nho.csv', index=False)